In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import os

In [42]:
# read in tsv file output from mmseqs

clusterDataFile = '/home/allen/projects/bsp/cluster/DB_05.tsv'

############################################################################
cdata = pd.read_csv(clusterDataFile,sep='\t', names=['cluster','id'])
print(cdata)
print(len(cdata.cluster))
print(len(cdata.cluster.unique()) )

            cluster    id
0             AccII     5
1          AchA6III     6
2       M.Ade56712I     7
3      S.Aje13652II    10
4        S.AmaHL49I    11
...             ...   ...
10809    Cginov21II  9185
10810   S.Cho10426I  9186
10811   S.Cho10426I  7104
10812     M.Cje328I  9188
10813     M.Cje328I  1769

[10814 rows x 2 columns]
10814
4235


In [43]:

clusterSet = set(cdata.cluster)
print(len(clusterSet))

4235


In [49]:
# read data 

dataDir = '../data'
csvFile = 'protein_seqs_cleaned_6types.csv'

dataDf = pd.read_csv( os.path.join( dataDir, csvFile ) )

print(dataDf)
print(dataDf.describe())
print(len(dataDf))
print(len(dataDf['RE'].unique()))
RESet = set(dataDf['RE'])

                 RE                        type            site  \
0        M.Aac9709I   Type II methyltransferase            GATC   
1          M.AacDam   Type II methyltransferase            GATC   
2        M.AalSMS7I  Type III methyltransferase          TGAATC   
3       M.AalSMS7II    Type I methyltransferase     AGCNNNNRTCA   
4       S.AalSMS7II  Type I specificity subunit     AGCNNNNRTCA   
...             ...                         ...             ...   
10809       M.ZmoII    Type I methyltransferase  GAAGNNNNNNNTCC   
10810       S.ZmoII  Type I specificity subunit  GAAGNNNNNNNTCC   
10811   M.Zmo29192I    Type I methyltransferase   ACANNNNNNRTGG   
10812   S.Zmo29192I  Type I specificity subunit   ACANNNNNNRTGG   
10813  M.Zmo29192II   Type II methyltransferase          GATATC   

                                                sequence  
0      MPEPAKPATPAKSRPFLKWAGGKYRLMDEINRLLPKRKQCLVEPFV...  
1      MPEPAKPATPAKSRPFLKWAGGKYRLMDEINRLLPKRKQCLVEPFV...  
2      MSKEKETTENP

In [20]:
nameInSet = dataDf['RE'].map( lambda x: x in clusterSet )

In [22]:
np.array(nameInSet).sum()

4204

In [ ]:
# now create some selections of sub data bases
# set the min-seq-id
min_seq_id = 0.5

clusterPickFraction = 0.15

#######################################################################

picks = np.random.choice([True, False],numberClusters,p=[clusterPickFraction,1-clusterPickFraction])
remaining = ~picks

pickGroupsSizes = groupCounts[ picks ]
remainingGroupsSizes = groupCounts[ remaining ]

print('\npick groups:\n',pd.Series(pickGroupsSizes).describe())
print('\nremaining groups:\n',pd.Series(remainingGroupsSizes).describe())

In [ ]:
# let take a data set made of representative sequence from each cluster

representativePicks =  groupNames[ picks ]
representativeRemaining = groupNames[ remaining ]

print('\nnumber of sequences:', numberSequences )
print('number of clusters:', numberClusters )
print('number of picked sequences:', len(representativePicks), len(representativePicks)/numberSequences )
print('number of remaining sequences:', len(representativeRemaining), len(representativeRemaining)/numberSequences )

        

In [ ]:
0.05724061401886443/(0.3343813574995376+0.05724061401886443)

In [ ]:
# now how do I make sure site space is covered?

In [ ]:
# save it here
# output file names --- 'None' if not saving reformated data

from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

combinedFileOutput = None # 'protein_seqs_cleaned_6types.csv'
fastaFileName = None #'protein_seqs_cleaned_6types.fasta'

if combinedFileOutput:
    temp[['RE', 'type', 'site','sequence' ]].to_csv(combinedFileOutput,index=False)

if fastaFileName:     # ---- Convert to SeqRecord objects ----
    records = []
    for _, row in temp.iterrows():
        seq = Seq(str(row["sequence"]).strip())
        records.append(
                    SeqRecord(seq, id=str(row["RE"]), description=row.RE+' '+row.site+' '+row.type)
                    )
        
    # ---- Write to FASTA ----
    SeqIO.write(records, fastaFileName, "fasta")